In [1]:
import json

data_dir="new_corpus/data/all_data"

with open(data_dir+"/val_data.json",'r') as file:
    sample_list=json.load(file)

In [2]:
len(sample_list),type(sample_list)

(14691, list)

In [3]:
# sample_list[25] -> This is one sample, which is a list of five items [tokens,token_tags,intent,slot_info,dialogue_context]

# let s be one sample
# s[0] -> list of words for the utterance of first sample.
# s[1] -> A list of values which indicates whether the word in that index is associated with any slot value
# s[2] -> List of intents for this utterance.
# s[3] -> List of slots for this utterance
# s[4] -> this is dialogue context or history
sample_list[25] 

[['I',
  'am',
  'also',
  'looking',
  'for',
  'a',
  'multi',
  'sport',
  'in',
  'the',
  'East',
  '.'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-Attraction-Inform+Area',
  'O'],
 [],
 [['Attraction-Inform', 'Area', 'east']],
 ['The earliest after 18:45 is the TR8658 , leaving Norwich at 19:16 . Can I reserve you one or more seats on this train ?',
  'yeah , i need one ticket',
  'Booking was successful , the total fee is 17.6 GBP payable at the station . Your reference number is AXH1NM1I . Do you need assistance with anything else ?']]

In [8]:
sample_list[25][1]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-Attraction-Inform+Area',
 'O']

In [10]:
with open(data_dir+"/intent_vocab.json",'r') as file:
    intent_voc=json.load(file)

In [11]:
intent_voc

['Taxi-Request',
 'general-thank',
 'general-reqmore',
 'general-bye',
 'Restaurant-Inform',
 'Restaurant-Request',
 'Attraction-Request',
 'Restaurant-Select',
 'Booking-Inform',
 'Booking-NoBook',
 'Booking-Request',
 'Taxi-Inform',
 'Train-Request',
 'general-welcome',
 'Hotel-Inform',
 'Train-OfferBook',
 'Hotel-Request',
 'Train-Inform',
 'general-greet',
 'Attraction-Inform',
 'Hotel-Recommend',
 'Attraction-Select',
 'Hotel-Select',
 'Train-Select',
 'Hotel-NoOffer',
 'Restaurant-NoOffer']